In [1]:
!pip install -r requirements.txt

In [2]:
#lst = list(await wiki_bench.client.client.models.list())[0][1]
#for l in lst:
#    print(l)

NameError: name 'wiki_bench' is not defined

In [1]:
import sys

sys.path.append("src")
from wiki_bench import WikiBench
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()

resource module not available on Windows


In [2]:
import logging
import torch
from sentence_transformers import SentenceTransformer

logging.getLogger("sentence_transformers.SentenceTransformer").setLevel(logging.ERROR)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = SentenceTransformer("sergeyzh/BERTA").to(device)

In [3]:
bench = WikiBench(
    url, key,
    model_name='ruadapt-qwen3-4b', # MODEL NAME
    device=device,
    encoder=encoder,
    number_of_articles=3
)

13:24:05 | INFO | wikibench | WikiBench initialized: model=ruadapt-qwen3-4b, articles=3


In [4]:
bench.load_enviroment()

13:24:05 | INFO | wikibench | Loading enviroment...
13:24:05 | INFO | wikibench | Enviroment loaded!


In [8]:
await bench.rank_query()

13:07:35 | INFO | wikibench | Stage: rank_query started
rank_query: 100%|███████████████████████████████████████████████████████████████████| 3/3 [01:18<00:00, 26.12s/article]
13:08:53 | INFO | wikibench | Final result for stage rank_query: (0.9759330529764757, 0.8759244535312232)


(0.9759330529764757, 0.8759244535312232)

In [5]:
await bench.rank_outline(
    neighbor_count=0,
    description_mode=0,
    clusterization_with_hint=True
)

13:24:05 | INFO | wikibench | Stage: rank_outline started: neighbor_count=0, description_mode=0, clusterization_with_hint=True
rank_outline: 100%|█████████████████████████████████████████████████████████████████| 3/3 [00:27<00:00,  9.15s/article]
C:\Users\dagri\anaconda3\envs\WIKIbench\Lib\site-packages\scipy\stats\_resampling.py:155: RuntimeWarning: invalid value encountered in scalar divide
  alpha_1 = ndtr(z0_hat + num1/(1 - a_hat*num1))
C:\Users\dagri\anaconda3\envs\WIKIbench\Lib\site-packages\scipy\stats\_resampling.py:157: RuntimeWarning: invalid value encountered in scalar divide
  alpha_2 = ndtr(z0_hat + num2/(1 - a_hat*num2))
C:\Users\dagri\WIKIBENCH\src\wiki_evaluater.py:134: DegenerateDataWarning: The BCa confidence interval cannot be calculated. This problem is known to occur when the distribution is degenerate or the statistic is np.min.
  bootstrap_ci = bootstrap(data1, np.mean, confidence_level=0.95, n_resamples=len(data)-1)
13:24:32 | INFO | wikibench | Final result for

(np.float32(0.45178393),
 np.float32(0.45178393),
 np.float32(0.45178393),
 np.float32(0.64656454),
 np.float32(0.6336606),
 np.float32(0.6594685),
 np.float32(0.57478356),
 np.float32(0.5480953),
 np.float32(0.60147184))

In [6]:
await bench.rank_sections()

13:24:39 | INFO | wikibench | Stage: rank_sections started
rank_sections: 100%|████████████████████████████████████████████████████████████████| 3/3 [00:57<00:00, 19.26s/article]
13:25:36 | INFO | wikibench | Final result for stage rank_sections: P=0.5117 [0.4831; 0.5569] | R=0.6098 [0.5749; 0.6704] | F=0.5497 [0.5177; 0.5856] | Rouge=0.5678 [0.5357; 0.6129] | BLEU=0.5559 [0.5026; 0.5899]


(np.float32(0.5116583),
 np.float32(0.48305404),
 np.float32(0.55691975),
 np.float32(0.6098494),
 np.float32(0.57491106),
 np.float32(0.67035115),
 np.float32(0.54965556),
 np.float32(0.51771057),
 np.float32(0.5856333),
 np.float32(0.5677792),
 np.float32(0.5356974),
 np.float32(0.6128838),
 np.float32(0.5558759),
 np.float32(0.50263613),
 np.float32(0.5898624))